## Long Short Distance Extraction from Unstructured Text

This experiment uses LLMs to extract distance related information from unstructured climate mobility literature.

### Libraries

In [1]:
from openai import OpenAI, RateLimitError, APIError, APITimeoutError, AuthenticationError, BadRequestError, NotFoundError
from google.api_core.exceptions import ResourceExhausted, RetryError, DeadlineExceeded
import google.generativeai as genai

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# from multiprocessing import Pool
import time
import os
import ast
from tqdm import tqdm

### Layout

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

### Data

In [18]:
literature = np.load('literature.npy', allow_pickle=True).item()
shot = np.load('example.npy', allow_pickle=True).item()

### Pre-defined variables

In [4]:
MAX_TOKEN = 16

### General commands for getting response

In [43]:
def api_def(provider):
    # API keys are hidden in another file
    with open('api.txt', 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines()]

        # Different LLMs need different kinds of command
        if provider == 'chatgpt':
            client = OpenAI(api_key=str(lines[0]), base_url="https://api.openai.com/v1")
            return client
        
        elif provider == 'deepseek':
            client = OpenAI(api_key=str(lines[1]), base_url="https://api.deepseek.com")
            return client
        
        elif provider == 'gemini':
            # My project with api in line 2 exceeds usage limit.
            genai.configure(api_key=str(lines[3]))

In [6]:
def get_completion(prompt, provider, model, temperature):
    messages = [{"role": "system", "content": "You are an expert in Human Geography and are reviewing papers. \
    Your task is to read a piece of provided text and simply classify it according to the prompt. Please answer the question without deep thinking"},
               {"role": "user", "content": prompt}]

    # Different LLMs need different kinds of command
    if provider == 'gpt':
        client = api_def('chatgpt')
        response = client.responses.create(
            model=model,
            input=messages,
            temperature=temperature,
            max_output_tokens = MAX_TOKEN
        )
        return response.output_text
        
    elif provider == 'ds':
        client = api_def('deepseek')
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens = MAX_TOKEN
        )
        return response.choices[0].message.content

    elif provider == 'gemini':
        api_def('gemini')
        model_gemini = genai.GenerativeModel(model)
        response = model_gemini.generate_content(
            prompt,
            generation_config={"temperature": temperature, "max_output_tokens": 100}
        )
        return response.text        

In [9]:
CONTEXT = f"""
We are reviewing a series of papers regarding whether they discuss 'long distance human migration', 'short distance human migration', or both of them, \
or even none of them. For each paper, give only one character as result: If it discusses 'long distance human migration', return '1'; If it discusses \
'short distance human migration', please return '2'; If it discusses both 'long distance human migration' and 'long distance human migration', return '3', \
while if it discusses nothing about distance, please return '0'. \n

While classifying a paper, consider only the content of this exact paper. Do not use information from elsewhere. Do not infer missing information from \
past tasks or papers. Focus primarily on the abstract, data/methods, results, and conclusions sections. \
Do not rely on introduction, state-of-the-art, related works, or motivation sections, especially when they describe other researchers’ work. \
Only use information from those sections if they explicitly describe what this paper does. \
Do not output anything else such as explanations, opinions or meta-comments. 
"""

In [21]:
def get_response(paper: dict, example, provider, model, temperature):

    fail_count = 0
    while True:
        if example == 3:
            prompt = f"""
            {CONTEXT} \n
            <Group leader>: Please assign a character for the following paper: \n
            \"\"\"{shot[0]}\"\"\"
            \n
            <Expert>: My answer is: 1 \n
            <Group leader>: Please assign a character for another paper: \n
            \"\"\"{shot[1]}\"\"\"
            \n
            <Expert>: My answer is: 3 \n
            <Group leader>: Please assign a character for another paper: \n
            \"\"\"{shot[2]}\"\"\"
            \n
            <Expert>: My answer is: 2 \n
            
            <Group leader>: Here is a new paper, please classify it like the expert: \n"
            f"{paper}\n"
            "Keep in mind that output is only one character."
            """

        elif example == 1:
            prompt = f"""
            {CONTEXT} \n
            <Group leader>: Please assign a character for the following paper: \n
            \"\"\"{shot[0]}\"\"\"
            \n
            <Expert>: My answer is: 1 \n
            
            <Group leader>: Here is a new paper, please classify it like the expert: \n"
            f"{paper}\n"
            "Keep in mind that output is only one character."
            """

        elif example == 0:
            prompt = f"""
            {CONTEXT} \n
            <Group leader>: Please assign a character for the following paper according to the requirement: \n
            f"{paper}\n"
            "Keep in mind that output is only one character."
            """

        response = get_completion(prompt, provider, model, temperature)
        return response

### Organization

In [44]:
def QA_to_df(literature, example, provider, model, idx):
    results = []
    fail_count = 0
    temperatures = np.round(np.arange(0.0, 2.0, 0.4), 1)
        
    for t in tqdm(temperatures, desc=f"File {idx+1}", leave=True):        
        for run_id in range(10):
            success = False
            while not success:
                try:
                    result = {"answer": get_response(literature, example, provider, model, t)}
                    result["provider"] = provider
                    result["model"] = model
                    result["few shot"] = example
                    result["temperature"] = t
                    result["run"] = run_id
                    results.append(result)
                    time.sleep(1)
                    success = True
                except (RateLimitError, APIError, APITimeoutError, ResourceExhausted) as e: # AuthenticationError, BadRequestError, NotFoundError
                    time.sleep(1)
                    fail_count = fail_count + 1
                    if fail_count >= 10:
                        raise ValueError(f"Can't get response, retried too many times") from e

    df = pd.DataFrame(results)
    return df

In [15]:
def batch_extract_to_df(all_literature, example, provider, model):

    save_path = f"results_s/{provider}_{example}.csv"
    os.makedirs("results_s", exist_ok=True)

    if os.path.exists(save_path):
        temporal_df = pd.read_csv(save_path)
        processed_ids = set(temporal_df["file_id"].astype(int).tolist())
        print(f"Start from paper {len(processed_ids)+1}")
    else:
        temporal_df = pd.DataFrame()
        processed_ids = set()
        
    all_results = [temporal_df] if not temporal_df.empty else []
    
    for idx, literature in enumerate(tqdm(all_literature.values(), desc="Progress", leave=False)):
        file_id = idx + 1
        if file_id in processed_ids:
            continue
        
        df = QA_to_df(literature, example, provider, model, idx)
        df["file_id"] = file_id
        all_results.append(df)

        temporal_df = pd.concat(all_results, ignore_index=True)
        temporal_df.astype(str).to_csv(save_path, index=False)

    final_df = pd.concat(all_results, ignore_index=True)
    return final_df

30 documents, 3 models, 3 prompt types, 5 temperatures, 10 runs, in sum 13500 api calls.

### Results

In [16]:
gpt41_0 = batch_extract_to_df(literature, 0, 'gpt', 'gpt-4.1-mini')

File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:52<00:00, 10.59s/it]
                                                                                                                       

In [20]:
gpt41_1 = batch_extract_to_df(literature, 1, 'gpt', 'gpt-4.1-mini')

File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:46<00:00,  9.24s/it]
                                                                                                                       

In [22]:
gpt41_3 = batch_extract_to_df(literature, 3, 'gpt', 'gpt-4.1-mini')

File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [01:02<00:00, 12.44s/it]
                                                                                                                       

In [23]:
ds32_0 = batch_extract_to_df(literature, 0, 'ds', 'deepseek-chat')

File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [01:23<00:00, 16.73s/it]
                                                                                                                       

In [32]:
ds32_1 = batch_extract_to_df(literature, 1, 'ds', 'deepseek-chat')

Start from paper 16


File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [01:24<00:00, 16.81s/it]
                                                                                                                       

In [28]:
ds32_3 = batch_extract_to_df(literature, 3, 'ds', 'deepseek-chat')

Start from paper 27


File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [01:43<00:00, 20.67s/it]
                                                                                                                       

In [29]:
gemini25_0 = batch_extract_to_df(literature, 0, 'gemini', 'gemini-2.5-flash')

File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [01:12<00:00, 14.44s/it]
                                                                                                                       

In [30]:
gemini25_1 = batch_extract_to_df(literature, 1, 'gemini', 'gemini-2.5-flash')

File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [01:13<00:00, 14.78s/it]
                                                                                                                       

In [45]:
gemini25_3 = batch_extract_to_df(literature, 3, 'gemini', 'gemini-2.5-flash')

File 30: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [02:05<00:00, 25.04s/it]
                                                                                                                       